<a href="https://colab.research.google.com/github/takumigit137/ss3/blob/main/SS3_Takumi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/openai/whisper.git


In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install pydub
from pydub import AudioSegment
import os
import whisper
!pip install pandas
import requests
import pandas as pd
import json


In [ ]:
# 音声データを4秒ずつに区切る
def split_and_trim_wav(input_path, output_path):
    # フォルダが存在しない場合は作成
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # 入力ファイルを読み込む
    sound = AudioSegment.from_wav(input_path)

    # 分割とトリミング
    duration = len(sound)
    segment_length = 4000  # 4秒（ミリ秒）
    start_time = 0

    for i in range(0, duration, segment_length):
        end_time = start_time + segment_length
        if end_time > duration:
            end_time = duration

        # 1秒ごとにトリミングして保存
        segment = sound[start_time:end_time]
        output_file = os.path.join(output_path, f'segment_{i // segment_length + 1}.wav')
        segment.export(output_file, format="wav")

        # 次のセグメントの開始時刻を更新
        start_time = end_time

In [ ]:
# 感情分析を行う関数
def analyze_emotion(file_path):
    payload = {'apikey': apikey}
    data = open(file_path, 'rb')
    file = {'wav': data}
    res = requests.post(url, params=payload, files=file)
    return res.json()

In [ ]:
def create_timelist(segments):
  timelist = []
  for i in range(len(segments)):
    timelist.append(segments[i]["end"])
  return timelist

In [ ]:
#盛り上がった４秒間の中間の２秒の時刻をリスト化
def joy_times(joy_list):
  joy_times_list = []
  for joy_index in joy_list:
    joy_times_list.append((joy_index+1)*4-2)

In [ ]:
def match_joy_sentence(joy_threshold,timelist,text_result,result_df):
  joy_sentence_list = []
  #感情が閾値の時のインデックスをリスト化
  joy_index_list = result_df[result_df["joy"] > joy_threshold].index
  time_index = 0
  skip_time_index = 0

  for joy_index in joy_index_list:
    dict = {}
    #盛り上がった４秒間のうち中間の２秒の時刻をリスト化
    joy_time = (joy_index+1)*4-2
    time_index = skip_time_index
    #盛り上がった時間がどこか見つけてその時のsegment_idとjoyの値とtextを
    while time_index < len(timelist):
      if joy_time <= timelist[time_index]
        dict["segment_id"] = time_index
        dict["joy"] = result_df["joy"][joy_index]
        dict["text"] = text_result["segments"][time_index]["text"]
        joy_sentence_list.append(dict)
        skip_time_index = time_index
        break
      else:
        time_index += 1
    #joyの値でソート
  return joy_sentence_list = sorted(joy_sentence_list,key=lambda x: x["joy"],reverse=True)

In [ ]:
# 入力ファイルと出力フォルダのパスを指定
input_wav_path = "/content/drive/MyDrive/SS3/test.wav"
output_folder_path = "/content/drive/MyDrive/SS3/test_output_segments"
# 音声データを感情分析して，データフレーム型として保存する
url = 'https://api.webempath.net/v2/analyzeWav'
#apikey = 'API'
input_folder = "/content/drive/MyDrive/SS3/test_output_segments"
output_csv_path = "/content/drive/MyDrive/SS3/test_emotion_results.csv"


In [ ]:
#zoomの音声ファイルをwav(16bitのPCM wave形式、サンプリング周波数は11025Hz、チャンネル数はモノラル)にする処理

In [ ]:
#文字起こし
model = whisper.load_model("base")

#結果は辞書型
#text_result["text"]に文字列全文が入ってる
#text_result["segments"]がリストになってる
#リストの各要素がid,text,start(各部分の開始時間),end(各部分の終了時間)の情報持つ辞書型
text_result = model.transcribe(input_wav_path)
all_text = text_result["text"]
timelist = create_timelist(text_result["segments"])

In [ ]:
# 分割とトリミングの実行
split_and_trim_wav(input_wav_path, output_folder_path)

In [ ]:
# データフレームを初期化
result_df = pd.DataFrame()

# 指定したフォルダ内のすべての.wavファイルに対して感情分析を実行
for filename in os.listdir(input_folder):
    if filename.endswith(".wav"):
        wav_path = os.path.join(input_folder, filename)
        result = analyze_emotion(wav_path)

        # 結果をデータフレームに追加
        df = pd.DataFrame([result])
        result_df = result_df.append(df, ignore_index=True)

        print(f"Analysis for {filename} completed. Result added to data frame.")

# データフレームをCSVファイルに保存
result_df.to_csv(output_csv_path, index=False)

print(f"CSV file saved at {output_csv_path}")

In [ ]:
#csvファイル読み込んでpandasにする
result_df = pd.read_csv(output_csv_path)
#joy_sentences:文と閾値以上の感情を対応させた(一応{segment_id,joy,text}の辞書型のリスト)
joy_threshold = 30
joy_sentences = match_joy_sentence(joy_threshold,timelist,text_result,result_df)

In [ ]:
#話題分割
!pip install tiktoken
!pip install cohere
!pip install openai

In [ ]:
#セグメントの区切りに「#」を挿入した文字列を作成
text_result_segments= text_result["segments"]
segments_text = text_result_segments[0]["text"]

for i in range(len(text_result_segments)-1):
  segments_text= segments_text +"#"+ text_result_segments[i+1]["text"]
segments_text = segments_text + "#"
segments_text = "[文章]" + segments_text

In [ ]:
#全文を話題毎に分ける
#segmentで分けた文字列をに対して話題分割
#responseにapiの応答が格納される
from openai import OpenAI
client = OpenAI(
  organization='org',api_key='API'
)
response = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[
    {"role": "system", "content": "あなたは優秀な会話分析官です。"},
    {"role": "user", "content": "[文章]は2人の会話です。[文章]は会話の切れ目で「#」が挿入されています。話題が変わるタイミングで「#」の後に「/」を挿入してください。また、話題の数は極力少なくしてください。"},
    {"role": "user", "content":segments_text}
]
)


In [ ]:
#各話題に含まれるsegment_idのリストを作る関数
def make_theme_id_list(string_array):
    # 各文字列の中の "#" の個数を格納するリスト
    counts = []
    all_hash_count = 0

    # 各文字列に対して処理
    for my_string in string_array:
        # "#" の個数を数えてリストに追加
        hash_count = my_string.count("#")
        counts.append(list(range(all_hash_count,all_hash_count + hash_count)))
        all_hash_count =all_hash_count + hash_count

    return counts

In [ ]:
split_text=response.choices[0].message.content.split("/") # 要素に各話題を持つ配列
theme_ids=make_theme_id_list(split_text) #各話題に含まれるsegment_idのリスト [[0,1],[2,3,4],・・・]

In [ ]:
def match_joy_theme(theme_num,theme_id,joy_sentences):
  theme_count = 0
  joy_theme_ids = []
  while theme_count < theme_num:
    for i in range(len(theme_id)):
      theme_candidate = theme_id[i]
      for j in range(len(thme_id[i])):
        if (theme_id[i][j] == joy_sentences[joy_count]["segment_id"]) and (theme_candidate not in joy_theme_ids):
          joy_theme_ids.append(theme_id[i])
          theme_count += 1
          break

  return joy_theme_ids

In [ ]:
#閾値以上の感情持ってる文がどの話題に属するか特定する
#joy_sentences:文と閾値以上の感情を対応させた({segment_id,joy,text}の辞書型のリスト)
#持ってくる話題の数
theme_num = 1
#joy_theme_ids:閾値以上の感情持ってる文が属している話題のid
joy_theme_ids = match_joy_theme(theme_num,theme_ids,joy_sentences)

In [ ]:
#話題を単語にする(単語と感情の大きさがセット)ver2
#入力：文章（変数topic_sentenceにstrで入力）, 出力：単語（topic_wordにstrで入力）
#response_topicにapiの応答が格納される
#感情については単語側での使い道が分からなかったので，とりあえずスルーして単語だけ出すことにしました

from openai import OpenAI

#topic_sentences:話題の配列(感情の大きい順)[話題A,話題B,話題C...]
topic_sentences = []
#topic_words:話題を表す単語の配列(感情の大きい順)
topic_words = []

for theme_id in theme_ids:
  topic_sentences.append(split_text[theme_id])
  #topic_sentence="浅野ともみです。今日の東京株式市場 で、日経平均株価は小幅続伸となっています 。終値は昨日に比べ二十二円七十二銭高の一万一千八十八円五十八銭でした。 "
  topic_sentence = split_text[theme_id]
  client = OpenAI(
    organization='org',api_key='API'
  )
  response_topic = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
      # {"role": "system", "content": "あなたは優秀な会話分析官です。"},
      # {"role": "user", "content": "[文章]は2人の会話です。この会話の内容を1単語にまとめてください．"},
      {"role": "system", "content": "あなたは優秀なデートプランナーです．"},
      {"role": "user", "content": "[文章]は2人の会話です。この会話の内容から，デートスポットを考えるのに使える具体性のある単語を1単語で答えてください"}
      {"role": "user", "content":topic_sentence}
  ]
  )
  print(response_topic.choices[0].message.content)
  topic_word = response_topic.choices[0].message.content
  #ChatGPTの応答文の長さを見て，10文字以下なら単語のリストに入れる
  if len(topic_word) <= 10:
    topic_words.append(topic_word)
print(topic_words)
#株価

In [ ]:
#単語からスポット提案
#入力：topic_words（list型でstr型の単語を格納）（topic_words=["中華","イルミネーション"]）
#出力：json_data0,json_data1,...（json型）
#単語に地名を追加してハッシュタグ化し，インスタのAPIで検索

#１．検索したいハッシュタグのIDを探す

import requests



for index_spot in range(len(topic_words)):
  # 検索したいワード
  query = "東京"+topic_words[index_spot]

  #query = "中華"
  #query = "東京"+query

  #IDとアクセストークン
  instragramID = "instragramID"
  ACCESS_TOKEN = "ACCESS_TOKEN"


  id_search_url = "https://graph.facebook.com/ig_hashtag_search?user_id=" + instragramID + "&q=" + query +  "&access_token=" + ACCESS_TOKEN

  response = requests.get(id_search_url)
  hash_id = response.json()
  print(hash_id)

  # ハッシュタグIDを確認
  # {'data': [{'id': '17843767135051777'}]}


  #２．検索タイプと取得したい項目を設定

  # 検索タイプを選択（今回は人気の投稿を探すやつ）
  serch_type = "top_media"
  hash_id2=hash_id["data"][0]
  if 'id' in hash_id2:
    hash_id3 = hash_id2['id']
  else:
    hash_id3= 'error'
  print(hash_id3)

  url = "https://graph.facebook.com/" + hash_id3 + "/" + serch_type + "?user_id=" + instragramID + "&q=" + query + "&access_token=" + ACCESS_TOKEN + "&fields=id,media_type,media_url,permalink,like_count,comments_count,caption,timestamp,children{id,media_url}&limit=10"
  #url = "https://graph.facebook.com/17843767135051777/?user_id=17841449273848018&q=マキアート&access_token=EAAcUTWw6gZBIBO5QxZB6ZAP0wgzpZBlzPiY5ZAkfIh2JAxo00gWv6jAy9zzNjP0Ef631Qaq8704K3HstYrXdUewC53eVuNOyuupYaHxRBq10X5eq6NxlM0Wqd1l2Kn1ZBsRGTBCkcNmGgK6Me4zhAdWtRQPQa2ZCbVlyFGVcgh5PkZBvZB6ytv3GdSQY3eq6OVUkPRQYZBHbQZD&fields=id,media_type,media_url,permalink,like_count,comments_count,caption,timestamp,children{id,media_url}&limit=50"

  #３．データをリクエスト
  response = requests.get(url)
  #json_data = response.json()
  name_json="json_data"+str(index_spot)
  globals()[name_json]=response.json()
#print(json_data["data"][0])


In [ ]:
# #4.　jsonの出力をcsvにする
# #まだ上のファイル名に対応できてなくて，変数dfに個別にファイル名を入れないといけない状況です．

# import pandas as pd
# import json
# from pandas.io.json import json_normalize

# #変換したいJSONファイルを読み込む
# df = json_data
# print(df)

# # read_jsonした結果だとネストしたjsonを展開できないのでnormalizeで展開させる
# df_json = json_normalize(df['data'])
# df_json.to_csv("out.csv", encoding='utf-8')

# print(df_json)



In [ ]:
##ChatGPTでデートプランを提案するプログラム
#入力：topic_words（list型でstr型の単語を格納）（topic_words=['中華','イルミネーション']）
#出力：spot_gpt（list型でstr型の単語を格納）（spot_gpt=['浅草', '六本木']）

# from openai import OpenAI


# #topic_words:話題を表す単語の配列(感情の大きい順)
# spot_gpt = []
# topic_words=["中華","イルミネーション"]
# for index_spot_gpt in range(len(topic_words)):
#   # 検索したいワード
#   #topic_sentence="浅野ともみです。今日の東京株式市場 で、日経平均株価は小幅続伸となっています 。終値は昨日に比べ二十二円七十二銭高の一万一千八十八円五十八銭でした。 "
#   topic_gpt = topic_words[index_spot_gpt]
#   client = OpenAI(
#     organization='org',api_key='API'
#   )
#   response_topic_gpt = client.chat.completions.create(
#     model="gpt-4-1106-preview",
#   #   messages=[
#   #     {"role": "system", "content": "あなたは優秀な会話分析官です。"},
#   #     {"role": "user", "content": "[文章]は2人の会話です。この会話の内容を1単語にまとめてください．"},
#   #     {"role": "user", "content":topic_sentence}
#   # ]
#     messages=[
#         {"role": "system", "content": "あなたは優秀なデートプランナーです。"},
#         {"role": "user", "content": "[文章]に関係する東京のデートスポットを1つ、簡潔に提案してください．"},
#         #{"role": "user", "content": "[文章]に関係する東京のデートスポットを1つ、一単語で提案してください．"},
#         {"role": "user", "content":topic_gpt}
#     ]
#   )
#   print(response_topic_gpt.choices[0].message.content)
#   spot_gpt.append(response_topic_gpt.choices[0].message.content)
# print(spot_gpt)